코드 참고 : https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Style_Transfer_Tutorial.ipynb

원본 : https://rn-unison.github.io/articulos/style_transfer.pdf

딥러닝을 활용한 스타일 전송 방법을 처음으로 제안
- 이미지 자체를 업데이트하는 이미지 최적화 기법

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models

import PIL
import matplotlib.pyplot as plt

import copy

In [ ]:
#GPU 장치 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#이미지를 불러와 다운받아 Tensor 객체로 변환

def imgage_loader(img_path, imgsize) :
  loader=transforms.Compose([
      transforms.Resize(imgsize)
      transforms.ToTensor()
  ])

  image = PIL.Image.open(img_path)
  #네트워크 입력에 들어갈 이미지에 배치 목적의 차원 추가
  image=loader(image).unsqueeze(0)
  return image.to(device, torch.float)    #gpu로 올리기

#torch.Tensor 형태의 이미지를 화면에 출력
def imshow(tensor) :
  #matplotlib는 cpu기반임으로 cpu로 옮기기
  image = tensor.cpu().clone()
  #torch.Tensor에서 사용되는 배치 목적의 차원 제거
  image = image.squeeze(0)
  #PIL객체로 변경
  image = transforms.ToPILImage(image)
  #이미지를 화면에 출력
  plt.imshow(image)
  plt.show()

In [ ]:
#저장소의 소스코드를 다운로드 하여 실습용 이미지 준비
!git clone https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice
%cd Deep-Learning-Paper-Review-and-Practice

#Image Reconstruction

- 이미지를 최적화 한다는 것은 특정 손실 값을 낮추는 방향으로 이미지를 업데이트
- MSE Loss를 이용하여 임의의 noise를 특정한 이미지로 변환

In [ ]:
#목표 이미지 불러오기
img_path = './code_practices/images/cat.jpg'
target_image = image_loader(img_path, (512,512))
imshow(target_image)

In [ ]:
#동일한 크기의 노이즈 이미지 준비
noise=torch.empty_like(target_image).uniform_(0,1).to(device)
imshow(noise)

In [ ]:
#이미지 업데이트

loss = nn.MSELoss()
iters = 100
lr =  1e4

print('[Start]')
imshow(noise)

for i in range(iters) :
  #required_grad 속성의 값을 True로 설정하여 해당 torch.Tensor의 연산을 추적
  noise.requires_grad = True

  #손실 함수에 대해 미분하여 gradient 계산
  output = loss(noise, target_image)
  output.backward()

  #계산된 기울기를 이용하여 손실 함수가 감소하는 방향으로 업데이트
  gradient = lr*noise.grad

  #결과적으로 노이즈의 각 픽셀의 값이 [-eps,eps] 사이의 값이 되도록 자르기
  noise = torch.clamp(noise - grandient, min=0, max=1).detach()   #연산을 추적하는 것을 중단하기 위한 detach함수

  if (i+1) % 10 == 0 :
    print(f'[ step : {i+1}]')
    print(f'Loss : {output}')
    imshow(noise)

이미지 불러오기

In [ ]:
#content 이미지와 style 이미지 준비
content_img = image_loader('./code_practices/images/content_img_1.jpg', (512, 640))
style_img = image_loader('./code_practices/images/style_img_1.jpg', (512, 640))

print('[Content image]')
imshow(content_img)

print('[Style image]')
imshow(style_img)

CNN 네트워크 불러오기
- VGG 네트워크 사용

In [ ]:
#뉴럴 네트워크 모델을 불러오기
cnn = models.vgg19(pretrained=True).features.to(device).eval()    #evaluation의 목적은 분류가 아닌 features를 추출하기 위함
print(cnn)

In [ ]:
#입력 정규화를 위한 초기화
cnn_normalization_mean = torch.tensor([0.485,0.456,0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229,0.224,0.225]).to(device)

class Normalization(nn.Module) :
  def __init__(self.mean, std) :
    super(Normalization, self).__init__()
    self.mean = mean.clone().view(-1,1,1)
    self.std = std.clone().view(-1,1,1)

  def forward(self, img) :
    return (img-self.mean) / self.std

# Style Reconstruction
- 임의의 noise가 특정한 이미지를 스타일을 갖도록 변환
- style_layer 리스트 변수의 값을 조절하여 어떤 레이어를 이용할지 설정 가능


In [ ]:
def gram_matrix(input) :
  #a는 배치의 크기, b는 특징 맵의 개수, (c,d)는 특징 맵의 차원
  a,b,c,d = input.size()
  features = input.view(a*b,c*d)
  # torch.mm :  matrix multiplication
  G = torch.mm(features, features.t())     #자기 자신과 전치를 행렬곱을 하게되면 gram matrix와 동일한 값이 출력됨 
  return G.div(a*b*c*d)

#Style loss계산을 위한 class 정의
class StyleLoss(nn.Module) :
  def __init__(self, target_feature) :
    super(StyleLoss, self).__init__()
    self.target = gram_matrix(target_feature).detach()

  def forward(self, input) :
    G = gram_matrix(input)
    self.loss = F.mse_loss(G, self.target)
    return input
  #style의 gram matrix를 구한 후, 나중에 들어온 input feature의 gram matrix를 구한 후 그 둘의 mse_loss

In [ ]:
style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

# 스타일 손실(style loss)을 계산하는 함수
def get_style_losses(cnn, style_img, noise_image):
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(cnn_normalization_mean, cnn_normalization_std).to(device)
    style_losses = []
    
    # 가장 먼저 입력 이미지가 입력 정규화(input normalization)를 수행하도록
    model = nn.Sequential(normalization)

    # 현재 CNN 모델에 포함되어 있는 모든 레이어를 확인하며
    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        # 설정한 style layer까지의 결과를 이용해 style loss를 계산
        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # 마지막 style loss 이후의 레이어는 사용하지 않도록
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]
    return model, style_losses

In [ ]:
def style_reconstruction(cnn, style_img, input_img, iters):
    model, style_losses = get_style_losses(cnn, style_img, input_img)
    optimizer = optim.LBFGS([input_img.requires_grad_()])   #현재 가장 많이 사용되는 optimizer는 Adam

    print("[ Start ]")
    imshow(input_img)

    # 하나의 값만 이용하기 위해 배열 형태로 사용
    run = [0]
    while run[0] <= iters:

        def closure():
            input_img.data.clamp_(0, 1)   #범위 제한

            optimizer.zero_grad()   #gradient 초기화
            model(input_img)
            style_score = 0

            for sl in style_losses:
                style_score += sl.loss

            style_score *= 1e6
            style_score.backward()

            run[0] += 1
            if run[0] % 50 == 0:
                print(f"[ Step: {run[0]} / Style loss: {style_score.item()}]")
                imshow(input_img)
            
            return style_score
        
        optimizer.step(closure)

    # 결과적으로 이미지의 각 픽셀의 값이 [0, 1] 사이의 값이 되도록 자르기
    input_img.data.clamp_(0, 1)

    return input_img

In [ ]:
# 콘텐츠 이미지와 동일한 크기의 노이즈 이미지 준비하기
input_img = torch.empty_like(content_img).uniform_(0, 1).to(device)
imshow(input_img)

In [ ]:
# style reconstruction 수행
output = style_reconstruction(cnn, style_img=style_img, input_img=input_img, iters=300)

#Content Reconstruction

임의의 노이즈가 특정한 이미지의 콘텐츠를 가지도록 변환

In [ ]:
# 콘텐츠 손실(content loss) 계산을 위한 클래스 정의
class ContentLoss(nn.Module):
    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

In [ ]:
#논문은 4번째 레이어와 input의 loss를 확인
content_layers = ['conv_4']

# 콘텐츠 손실(content loss)을 계산하는 함수
def get_content_losses(cnn, content_img, noise_image):
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(cnn_normalization_mean, cnn_normalization_std).to(device)
    content_losses = []
    
    # 가장 먼저 입력 이미지가 입력 정규화(input normalization)를 수행하도록
    model = nn.Sequential(normalization)

    # 현재 CNN 모델에 포함되어 있는 모든 레이어를 확인하며
    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        # 설정한 content layer까지의 결과를 이용해 content loss를 계산
        if name in content_layers:
            target_feature = model(content_img).detach()
            content_loss = ContentLoss(target_feature)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

    # 마지막 content loss 이후의 레이어는 사용하지 않도록
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss):
            break

    model = model[:(i + 1)]
    return model, content_losses

In [ ]:
def content_reconstruction(cnn, content_img, input_img, iters):
    model, content_losses = get_content_losses(cnn, content_img, input_img)
    optimizer = optim.LBFGS([input_img.requires_grad_()])

    print("[ Start ]")
    imshow(input_img)

    # 하나의 값만 이용하기 위해 배열 형태로 사용
    run = [0]
    while run[0] <= iters:

        def closure():
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            content_score = 0

            for cl in content_losses:
                content_score += cl.loss

            content_score.backward()

            run[0] += 1
            if run[0] % 50 == 0:
                print(f"[ Step: {run[0]} / Content loss: {content_score.item()}]")
                imshow(input_img)
            
            return content_score
        
        optimizer.step(closure)

    # 결과적으로 이미지의 각 픽셀의 값이 [0, 1] 사이의 값이 되도록 자르기
    input_img.data.clamp_(0, 1)

    return input_img

In [ ]:
# 콘텐츠 이미지와 동일한 크기의 노이즈 이미지 준비하기
input_img = torch.empty_like(content_img).uniform_(0, 1).to(device)
imshow(input_img)

In [ ]:
# content reconstruction 수행
output = content_reconstruction(cnn, content_img=content_img, input_img=input_img, iters=300)

# Style Transfer
- content Loss와 Style Loss를 함께 이용하여 Style Transfer 진행

In [ ]:
content_layers = ['conv_4']
style_layers = ['conv_1', 'conv_3', 'conv_5', 'conv_7', 'conv_9']

# Style Transfer 손실(loss)을 계산하는 함수
def get_losses(cnn, content_img, style_img, noise_image):
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(cnn_normalization_mean, cnn_normalization_std).to(device)
    content_losses = []
    style_losses = []
    
    # 가장 먼저 입력 이미지가 입력 정규화(input normalization)를 수행하도록
    model = nn.Sequential(normalization)

    # 현재 CNN 모델에 포함되어 있는 모든 레이어를 확인하며
    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

    #content와 style loss 모두 다 초기화 : 본문에서 동시에 업데이트하기 때문
    
        # 설정한 content layer까지의 결과를 이용해 content loss를 계산
        if name in content_layers:
            target_feature = model(content_img).detach()
            content_loss = ContentLoss(target_feature)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        # 설정한 style layer까지의 결과를 이용해 style loss를 계산
        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # 마지막 loss 이후의 레이어는 사용하지 않도록
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]
    return model, content_losses, style_losses

In [ ]:
def style_transfer(cnn, content_img, style_img, input_img, iters):
    model, content_losses, style_losses = get_losses(cnn, content_img, style_img, input_img)
    optimizer = optim.LBFGS([input_img.requires_grad_()])

    print("[ Start ]")
    imshow(input_img)

    # 하나의 값만 이용하기 위해 배열 형태로 사용
    run = [0]
    while run[0] <= iters:

        def closure():
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            content_score = 0
            style_score = 0

            for cl in content_losses:
                content_score += cl.loss
            for sl in style_losses:
                style_score += sl.loss

            style_score *= 1e5
            loss = content_score + style_score      #total loss
            loss.backward()

            run[0] += 1
            if run[0] % 100 == 0:
                print(f"[ Step: {run[0]} / Content loss: {content_score.item()} / Style loss: {style_score.item()}]")
                imshow(input_img)
            
            return content_score + style_score
        
        optimizer.step(closure)

    # 결과적으로 이미지의 각 픽셀의 값이 [0, 1] 사이의 값이 되도록 자르기
    input_img.data.clamp_(0, 1)

    return input_img

In [ ]:
# 콘텐츠(Content) 이미지와 스타일(Style) 이미지를 모두 준비합니다.
content_img = image_loader('./code_practices/images/content_img_1.jpg', (512, 640))
style_img = image_loader('./code_practices/images/style_img_1.jpg', (512, 640))

print("[ Content Image ]")
imshow(content_img)
print("[ Style Image ]")
imshow(style_img)

In [ ]:
# 콘텐츠 이미지와 동일한 크기의 노이즈 이미지 준비하기
input_img = torch.empty_like(content_img).uniform_(0, 1).to(device)
imshow(input_img)

In [ ]:
# style transfer 수행
output = style_transfer(cnn, content_img=content_img, style_img=style_img, input_img=input_img, iters=900)

In [ ]:
from torchvision.utils import save_image

save_image(output.cpu().detach()[0], 'output_1.png')
print('이미지 파일 저장을 완료했습니다.')

In [ ]:
from google.colab import files

# 완성된 이미지를 다운로드합니다.
files.download('output_1.png')

In [ ]:
from IPython.display import Image

Image('output_1.png')